# Programming Exercise 4: Neural Networks Learning

> In this exercise, you will implement the backpropagation algorithm for neural networks and apply it to the task of hand-written digit recognition.

## 1. Neural Networks

### 1.1 Generate Random Example Data


Loading the dataset:

In [1]:
import scipy.io
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

Visualizing the data:

In [2]:
SAMPLES = 6
INPUT_LAYER_SIZE = 5
HIDDEN_LAYER_SIZE = 2
OUTPUT_LAYER_SIZE = 2

In [3]:
mat = {'X':np.random.randint(10, size=(SAMPLES, INPUT_LAYER_SIZE))/10 , 'y': np.random.randint(OUTPUT_LAYER_SIZE, size=(SAMPLES,1))}
mat['X'].shape,mat['y'].shape

((6, 5), (6, 1))

In [4]:
mat['X']

array([[0.2, 0.1, 0.9, 0.1, 0.4],
       [0.3, 0.1, 0.5, 0.9, 0.4],
       [0.2, 0.3, 0.7, 0.8, 0.8],
       [0.5, 0.7, 0.7, 0.2, 0.7],
       [0.1, 0. , 0.1, 0.8, 0.8],
       [0.1, 0.5, 0.4, 0. , 0.7]])

In [5]:
mat['y']

array([[0],
       [0],
       [1],
       [1],
       [0],
       [1]])

### 1.2 Model representation

<img src="neural_network.png">

* $a_i^{(j)}$ = activation of unit $i$ in layer $j$

* $\Theta^{(j)}$ = matrix of weights controlling function mapping from layer $j$ to layer $j+1$. It's dimension is $s_{j+1}\times(s_j+1)$ where $s_j$ is the number of units in layer $j$ and $s_{j+1}$ is the number of units in layer $j+1$.

* $z^{(j+1)} = \Theta^{(j)}a^{(j)}$

* $h_\Theta(x) = a^{(j+1)} = g(z^{(j+1)}) $



Our neural network has:
* $3$ layers: an input layer, a hidden layer and an output layer
* $9$ input layer units (because the images are of size $3 \times 3$
* $3$ units in the second layer
* $2$ output units 
* $\Theta^{(1)}$ and $\Theta^{(2)}$ are provided
    * $\Theta^{(1)}$ has dimension $2 \times 6$
    * $\Theta^{(2)}$ has dimension $2 \times 3$


## $\Theta $

## The random parameters

In [18]:
mat_weights = {'Theta1': np.random.rand(HIDDEN_LAYER_SIZE, INPUT_LAYER_SIZE+1), 'Theta2': np.random.rand(OUTPUT_LAYER_SIZE, HIDDEN_LAYER_SIZE+1)}


In [19]:
print(mat_weights['Theta1'])
print(mat_weights['Theta2'])

[[0.65398817 0.64377754 0.26544022 0.60905877 0.92407062 0.31752954]
 [0.10812528 0.59723467 0.36889173 0.22614974 0.00799152 0.50631598]]
[[0.42844833 0.63090122 0.36733865]
 [0.57567602 0.24535538 0.80810448]]


Unroll parameters:

In [20]:
nn_params = np.hstack((mat_weights['Theta1'].ravel(order='F'), 
                       mat_weights['Theta2'].ravel(order='F')))

In [21]:
nn_params.shape

(18,)

### 1.3 Feedforward and cost function

Cost function without regularization

$$ J(\theta) = \frac{1}{m} \sum_{i=1}^m \sum_{k=1}^K [-y_k^{(i)} log((h_\theta(x^{(i)}))_k) - (1-y_k^{(i)})log(1-(h_\theta(x^{(i)}))_k)]$$

In [22]:
def sigmoid(z):
    z = np.array(z)
    return 1 / (1+np.exp(-z))

In [23]:
def nn_cost_function(nn_params, input_layer_size, hidden_layer_size, num_labels, X, y):
    
    theta1 = np.reshape(nn_params[:hidden_layer_size * (input_layer_size+1)], newshape=(hidden_layer_size, input_layer_size+1), order='F')
    theta2 = np.reshape(nn_params[hidden_layer_size * (input_layer_size+1):], newshape=(num_labels, hidden_layer_size+1), order='F')
    
    m = X.shape[0]
    J = 0
    
    K = num_labels
    X = np.hstack((np.ones((m,1)),X)) #add bias unit

    for i in range(m):
        a1 = X[i]
        
        z2 = a1.dot(theta1.T)
        a2 = sigmoid(z2)
        a2 = np.hstack([1, a2]) ##add bias unit
        
        z3 = a2.dot(theta2.T)
        a3 = sigmoid(z3)
        
        h = a3
        
        yk = np.zeros((K,1)) ##y is as K-dimensional vector
        yk[y[i,0]-1, 0] = 1
        
        j = (-yk.T.dot(np.log(h).T) - (1-yk).T.dot(np.log(1-h).T))
        J = J + (j/m)
    return J


In [24]:
J = nn_cost_function(nn_params, INPUT_LAYER_SIZE, HIDDEN_LAYER_SIZE, OUTPUT_LAYER_SIZE, mat['X'], mat['y'])
print(f'Cost at parameters: {J} \n')

Cost at parameters: [1.78069871] 



## 2. Backpropagation

> In this part of the exercise, you will implement the backpropagation algorithm to compute the gradient for the neural network cost function.

### 2.1 Sigmoid gradient

$$g'(z) = \frac{d}{dz}g(z) = g(z)(1-g(z))$$ 

where $g(z) = sigmoid(z) = \frac{1}{1+e^{-z}}$ 

In [12]:
def sigmoid_gradient(z):
    return sigmoid(z) * (1-sigmoid(z))

### 2.2 Random initialization
> When training neural networks, it is important to randomly initialize the pa- rameters for symmetry breaking. One effective strategy for random initialization is to randomly select values for $\Theta^{(l)}$ uniformly in the range $[−\epsilon_{init}, \epsilon_{init}]$. You should use $\epsilon_{init} = 0.12$.

In [13]:
epsilon_init = 0.12
initial_theta1 = np.random.uniform(low=-epsilon_init, high=epsilon_init, size=(hidden_layer_size, input_layer_size+1))
initial_theta2 = np.random.uniform(low=-epsilon_init, high=epsilon_init, size=(num_labels, hidden_layer_size+1))

NameError: name 'hidden_layer_size' is not defined

### 2.3 Backpropagation

> Given a training example $(x^{(t)},y^{(t)})$, we will first run a "forward pass" to compute all the activations throughout the network, including the output value of the hypothesis $h_\Theta(x)$. Then, for each node $j$ in layer $l$, we would like to compute an "error term" $\delta_j^{(l)}$ that measures how much that node was "responsible" for any errors in our output.

<img src="backpropagation.png">

Algorithm:

1) Set the input layer's values $(a^{(1)})$ to the t-th training example $x^{(t)}$.
Perform a feedforward pass, computing the activations ($z^{(2)}, a^{(2)}, z^{(3)}, a^{(3)}$) for layers 2 and 3.

2) For each output unit $k$ in layer $3$ (the output layer), set 

$$ \delta_k^{(3)} = (a_k^{(3)} - y_k)$$

3) For the hidden layer $l = 2$, set

$$ \delta_k^{(2)} = (\Theta^{(2)})^T \delta^{(3)} .* g'(z^{(2)})$$ 

4) Accumulate the gradient from this example using the following formula. Note that you should skip or remove $\delta_0^{(2)}$

$$ \Delta^{(l)} = \Delta^{(l)} + \delta^{(l+1)}(a^{(l)})^T$$

5) Obtain the (unregularized) gradient for the neural network cost func-
tion by dividing the accumulated gradients by $\frac{1}{m}$:

$$ D_{ij}^{(l)} = \frac{1}{m}\Delta_{ij}^{(l)}$$

In [ ]:
def nn_cost_function(nn_params, input_layer_size, hidden_layer_size, num_labels, X, y, lambda_r):
    
    theta1 = np.reshape(nn_params[:hidden_layer_size * (input_layer_size+1)], newshape=(hidden_layer_size, input_layer_size+1), order='F')
    theta2 = np.reshape(nn_params[hidden_layer_size * (input_layer_size+1):], newshape=(num_labels, hidden_layer_size+1), order='F')
    
    m = X.shape[0]
    J = 0
    
    K = num_labels
    X = np.hstack((np.ones((m,1)),X)) #add bias unit

    capital_delta1 = np.zeros(theta1.shape)
    capital_delta2 = np.zeros(theta2.shape)
    
    for i in range(m):
        a1 = X[i]
        
        z2 = a1.dot(theta1.T)
        a2 = sigmoid(z2)
        a2 = np.hstack([1, a2]) ##add bias unit
        
        z3 = a2.dot(theta2.T)
        a3 = sigmoid(z3)
        
        h = a3
        
        yk = np.zeros((K,1)) ##y is as K-dimensional vector
        yk[y[i,0]-1, 0] = 1
        
        j = (-yk.T.dot(np.log(h).T) - (1-yk).T.dot(np.log(1-h).T)) ##sum of K
        J = J + (j/m) #sum of i
        
        delta3 = a3 - yk.T
        
        z2 = np.hstack([1, z2])
        delta2 = theta2.T.dot(delta3.T) * (sigmoid_gradient(z2).reshape(-1,1))
        
        capital_delta1 = capital_delta1 + (delta2[1:,:].dot(a1.reshape(1,-1)))
        capital_delta2 = capital_delta2 + (delta3.T.dot(a2.reshape(1,-1)))
        
    sum1 = np.sum(np.sum(theta1[:, 1:] ** 2))
    sum2 = np.sum(np.sum(theta2[:, 1:] ** 2))
    J = J + (lambda_r / (2*m)) * (sum1 + sum2)
    
    theta1_grad = (1/m) * (capital_delta1 + lambda_r * theta1) #with regularization
    theta1_grad[:,0] = ((1/m) * capital_delta1)[:,0]
    
    theta2_grad = (1/m) * (capital_delta2 + lambda_r * theta2) #with regularization
    theta2_grad[:,0] = ((1/m) * capital_delta2)[:,0]
    
    grad = np.hstack((theta1_grad.ravel(order='F'), theta2_grad.ravel(order='F')))
    return J, grad

### 2.4 Gradient Checking

> In your neural network, you are minimizing the cost function $J(\Theta)$. To perform gradient checking on your parameters, you can imagine “unrolling” the parameters $\Theta^{(1)}, \Theta^{(2)}$ into a long vector $\theta$. By doing so, you can think of the cost function being $J(\theta)$ instead and use the following gradient checking procedure. Suppose you have a function $f_i(\theta)$ that purportedly computes $\frac{\partial}{\partial \theta_i} J(\theta)$; you’d like to check if $f_i$ is outputting correct derivative values. If $\theta^{(i+)}$ is the same $\theta$, except its i-th element has been incremented by $\epsilon$. You can now numerically verify $f_i(\theta)$’s correctness by checking, for each $i$, that: $f_i(\theta) 	\approx \frac{J(\theta^{(i+)} - J(\theta^{(i-)}}{2\epsilon}$. The degree to which these two values should approximate each other will depend on the details of $J$. But assuming $\epsilon = 10^{−4}$, you’ll usually find that the left- and right-hand sides of the above will agree to at least 4 significant digits (and often many more).

In [ ]:
def compute_numerical_gradient(theta, input_layer_size, hidden_layer_size, num_labels, X, y, lambda_r):
    e = 0.0001
    num_grad = np.zeros(theta.shape)
    perturb = np.zeros(theta.shape)
    for p in range(len(theta)):
        perturb[p] = e
        loss1, _ = nn_cost_function(theta-perturb, input_layer_size, hidden_layer_size, num_labels, X, y, lambda_r)
        loss2, _ = nn_cost_function(theta+perturb, input_layer_size, hidden_layer_size, num_labels, X, y, lambda_r)
        num_grad[p] = (loss2-loss1)/(2*e)
        perturb[p] = 0
    return num_grad

In [ ]:
def debug_initialize_weights(fan_out, fan_in):
    W = np.zeros((fan_out, 1+fan_in))
    W = np.reshape(range(len(W.ravel(order='F'))), W.shape)/10
    return W

In [ ]:
def check_nn_gradients(lambda_r=0):
    input_layer_size = 3
    hidden_layer_size = 5
    num_labels = 3
    m = 5
    
    theta1 = debug_initialize_weights(hidden_layer_size, input_layer_size)
    theta2 = debug_initialize_weights(num_labels, hidden_layer_size)
    
    X = debug_initialize_weights(m, input_layer_size-1)
    y = 1 + np.mod(range(m), num_labels).reshape(-1, 1)
    
    nn_params = np.hstack((theta1.ravel(order='F'), theta2.ravel(order='F')))
    
    cost, grad = nn_cost_function(nn_params, input_layer_size, hidden_layer_size, num_labels, X, y, lambda_r)
    num_grad = compute_numerical_gradient(nn_params, input_layer_size, hidden_layer_size, num_labels, X, y, lambda_r)
    
    print('The columns should be very similar...')
    for i, j in zip(num_grad, grad):
        print(i,j)
        
    diff = np.linalg.norm(num_grad-grad)/np.linalg.norm(num_grad+grad)
    if diff < 0.000000010:
        print('\nBackpropagation is correct')
    else:
        print('\nBackpropagation is incorrect')

In [ ]:
check_nn_gradients()

In [ ]:
lambda_r = 3
check_nn_gradients(lambda_r)
J, grad = nn_cost_function(nn_params, input_layer_size, hidden_layer_size, num_labels, mat['X'], mat['y'], lambda_r)
print('Cost at parameters (loaded from ex4weights): {0} \n(this value should be about 0.576051)'.format(J))

### 2.5 Learning Parameters - Training the Neural Network

In [ ]:
import scipy.optimize as opt
lambda_r = 1
opt_results = opt.minimize(nn_cost_function, nn_params, args=(input_layer_size, 
                                                              hidden_layer_size, 
                                                              num_labels, 
                                                              mat['X'], 
                                                              mat['y'], 
                                                              lambda_r), 
                            method='L-BFGS-B', jac=True, options={'maxiter':50})

In [ ]:
opt_results

In [ ]:
theta1 = np.reshape(opt_results['x'][:hidden_layer_size * (input_layer_size+1)], newshape=(hidden_layer_size, input_layer_size+1), order='F')
theta2 = np.reshape(opt_results['x'][hidden_layer_size * (input_layer_size+1):], newshape=(num_labels, hidden_layer_size+1), order='F')    

In [ ]:
def predict_nn(theta1, theta2, X):
    m, n = X.shape
    a1 = np.hstack((np.ones((m,1)),X)) #with a0
    
    z2 = a1.dot(theta1.T)
    a2 = sigmoid(z2)
    
    z3 = np.hstack((np.ones((m,1)),a2)).dot(theta2.T) #with a0
    a3 = sigmoid(z3)
    h = np.argmax(a3, axis=1)+1 #get label with largest h(x)
    
    return h

In [ ]:
y_pred = predict_nn(theta1, theta2, mat['X'])
accuracy = np.mean(y_pred == mat['y'].T)
f'Train accuracy: {accuracy * 100}'

#### 2.5.1 Similar Code using Scikit-Learn:

In [ ]:
from sklearn.neural_network import MLPClassifier
nn = MLPClassifier(hidden_layer_sizes=(hidden_layer_size,), activation='logistic', solver='lbfgs', alpha=1, max_iter=50)
nn.fit(mat['X'], mat['y'].T[0])

In [ ]:
nn.score(mat['X'], mat['y'].T[0])

## 3. Visualizing the hidden layer

In [ ]:
rows = 5
cols = 5
fig = plt.figure(figsize=(5,5))
indexes = np.random.choice(25, rows*cols)
count = 0
for i in range(0,rows):
    for j in range(0,cols):
        ax1 = fig.add_subplot(rows, cols, count+1)
        ax1.imshow(theta1[:,1:][indexes[count]].reshape((20,20), order='F'), cmap='gray')
        ax1.set_axis_off()
        count+=1
plt.subplots_adjust(wspace=.1, hspace=.1, left=0, right=1, bottom=0, top=1)
plt.show()

### 3.1 Trying Different Learning Settings

> Neural networks are very powerful models that can form highly complex decision boundaries. Without regularization, it is possible for a neural network to “overfit” a training set so that it obtains close to 100% accuracy on the training set but does not as well on new examples that it has not seen before.

In [ ]:
%%time
lambda_r = 0.1
opt_results = opt.minimize(nn_cost_function, nn_params, args=(input_layer_size, hidden_layer_size, num_labels, 
                                                              mat['X'], mat['y'], lambda_r), 
                            method='TNC', jac=True, options={'maxiter':100})
theta1 = np.reshape(opt_results['x'][:hidden_layer_size * (input_layer_size+1)], newshape=(hidden_layer_size, input_layer_size+1), order='F')
theta2 = np.reshape(opt_results['x'][hidden_layer_size * (input_layer_size+1):], newshape=(num_labels, hidden_layer_size+1), order='F')   
y_pred = predict_nn(theta1, theta2, mat['X'])
accuracy = np.mean(y_pred == mat['y'].T)

In [ ]:
f'Train accuracy with lambda={lambda_r} and maxiter=50: {accuracy * 100}'

In [ ]:
rows = 5
cols = 5
fig = plt.figure(figsize=(5,5))
indexes = np.random.choice(25, rows*cols)
count = 0
for i in range(0,rows):
    for j in range(0,cols):
        ax1 = fig.add_subplot(rows, cols, count+1)
        ax1.imshow(theta1[:,1:][indexes[count]].reshape((20,20), order='F'), cmap='gray')
        ax1.set_axis_off()
        count+=1
plt.subplots_adjust(wspace=.1, hspace=.1, left=0, right=1, bottom=0, top=1)
plt.show()

In [ ]:
theta1